# Import all packages

In [1]:
from __future__ import division 
import skimage
from skimage import io
from skimage import draw
from skimage.transform import rotate
import skimage.morphology
import scipy.ndimage as ndimage
import numpy as np
import subprocess
import os
import pandas as pd
import shutil
import random
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import networkx 
from networkx.algorithms.components.connected import connected_components
import re
import seaborn as sns
import math  
from matplotlib.patches import Ellipse
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import statistics


AttributeError: module 'matplotlib.pyplot' has no attribute 'use'

In [1]:
from pandas_profiling import ProfileReport

## 3D plot of network with mtDNA spots (not necessary for Dataframe)


In [ ]:
a, b, c = getFilamentFromVtk('D:\25TFSingleCells\200828yFT006_series1_000\Channel1\MitoGraphFiles'

In [ ]:
spot_c2 = getSpotCoordinates('200828yFT006_series3_000_T0_C=2.csv')
%matplotlib qt
fig = plt.figure()
ax2 = fig.add_subplot(111, projection='3d')
Axes3D.scatter(ax2, a[:,0], a[:,1], zs=a[:,2])
Axes3D.scatter(ax2, spot_c2[:,0], 22-spot_c2[:,1], zs=spot_c2[:,2], c='r')

# Create a list that contains all cells, and run mitograph over that list

In [4]:
root_folder = 'D:\WT_Singlecells'
List_of_Cells = []
for i in os.listdir(root_folder):
    if os.path.isdir(os.path.join(root_folder, i)):
        List_of_Cells.append(i)
#for movie in List_of_Cells:
    #runMitoGraph(os.path.join(root_folder, movie))

In [5]:
List_of_Cells

['201021_yFT006_series1_000',
 '201021_yFT006_series1_001',
 '201021_yFT006_series1_002',
 '201021_yFT006_series3_000',
 '201021_yFT006_series3_001',
 '201021_yFT006_series3_002',
 '201021_yFT006_series3_003',
 '201021_yFT006_series3_004',
 '201021_yFT006_series3_005',
 '201021_yFT006_series3_006',
 '201021_yFT006_series4_000',
 '201021_yFT006_series4_001',
 '201021_yFT006_series4_002',
 '201021_yFT006_series5_000',
 '201021_yFT006_series5_001',
 '201021_yFT006_series5_002',
 '201021_yFT006_series5_003',
 '201021_yFT006_series5_004',
 '201021_yFT006_series6_000',
 '201021_yFT006_series6_001',
 '201023_yFT006_series1_000',
 '201023_yFT006_series1_001',
 '201023_yFT006_series2_000',
 '201023_yFT006_series2_001',
 '201023_yFT006_series2_002',
 '201023_yFT006_series2_003',
 '201023_yFT006_series2_004',
 '201023_yFT006_series2_005',
 '201023_yFT006_series3_000',
 '201023_yFT006_series5_000',
 '201023_yFT006_series5_001',
 '201023_yFT006_series6_000',
 '201023_yFT006_series6_001',
 '201026_y

# Put all the data in one dataframe

In [15]:
results = {}
p = re.compile('_T(\d+)_')
all_cells = pd.DataFrame() 
for movie in List_of_Cells:
    
    tif_files = []
    for file in os.listdir(os.path.join(root_folder, movie, 'Channel1')):
        
        if file.endswith('.tif'):
            tif_files.append(file[:-5])
            print(file)
    for cell in tif_files:
        print(cell)
        results[cell] = {}
        fil_c, fil_e, segments = getFilamentFromVtk(os.path.join(root_folder,
                                                                 movie,
                                                                 'Channel1',
                                                                 'MitoGraphFiles',
                                                                 cell + '1_skeleton.vtk'))
        
        
        outline, volMother, volDaughter= getOutline(os.path.join(root_folder,
                                                                  movie,
                                                                  'Channel1',
                                                                  'Cell_outline/' + cell + '0.csv'))
        coordinates = getSpotCoordinates_mask(os.path.join(root_folder,
                                                                 movie,
                                                                 'Channel1',
                                                                 'SpotCoordinates',cell + '2.csv'))
        mask, mask_d = getEllipses(os.path.join(root_folder,
                                                movie,
                                                'Channel1',
                                                'Cell_outline/' + cell + '0.csv'))
        Network_Mother,Network_daughter, OutsideofMask = assignNetwork(fil_c, mask, mask_d)
        relNetworkMother, relNetworkDaughter = percentageNetwork(Network_Mother,Network_daughter, OutsideofMask)
        nucleoids_mother, nucleoids_daughter, OutsideOfMask, Intensities = assignNucleoids(coordinates, mask , mask_d)
        
        results[cell]['Coordinates'] = fil_c 
        results[cell]['Edges'] = fil_e
        results[cell]['Segments'] = len(segments) 
        results[cell]['SpotCoordinates'] = getSpotCoordinates(os.path.join(root_folder,
                                                                                  movie,
                                                                                  'Channel1',
                                                                                  'SpotCoordinates',
                                                                                  cell + '2.csv'))
        results[cell]['Number_of_Spots'] = int(results[cell]['SpotCoordinates'].shape[0])
        results[cell]['Nucleoids_in_Mother']= len(nucleoids_mother)
        results[cell]['Nucleoids_in_Daughter']= len(nucleoids_daughter)
        results[cell]['Nucleoids_in_both_cells']= results[cell]["Nucleoids_in_Mother"]+results[cell]["Nucleoids_in_Daughter"]
        results[cell]['Timepoint'] = p.findall(cell)[-1]
        results[cell]['Time']= int(results[cell]['Timepoint'])*7
        results[cell]['movie'] = movie
        results[cell]['Point_of_bud_emergence'] = int(getEmergencePoint(os.path.join(root_folder,
                                                                  movie,
                                                                  'Channel1', 
                                                                  'Bud_Emergence', 'BudEmergence.csv')))-1
        results[cell]['Timepoint_normalized']= (int(results[cell]['Timepoint']) - int(results[cell]['Point_of_bud_emergence']))*7
        results[cell]['Outline'] = outline
        results[cell]['volMother'] = volMother
        results[cell]['volDaughter'] = volDaughter
        results[cell]['Mean_spot_intensity'] = statistics.mean(Intensities)
        results[cell]['Volume_total']=results[cell]["volMother"]+results[cell]["volDaughter"]
        
        df2 = convertIntoPixelUnit(os.path.join(root_folder,
                                     movie,
                                    'Channel1',
                                    'MitoGraphFiles',
                                     cell))
        df2['Cells'] = defineCell(df2, mask, mask_d)
        df2['File'] = cell
        networklengths = {'Mother':calculateNetworkLength(df2[df2['Cells'] == 'Mother']),
                  'Daughter':calculateNetworkLength(df2[df2['Cells'] == 'Daughter']),
                         'Total_Network':calculateNetworkLength(df2)}
        result =  pd.DataFrame(networklengths, index=[0])
        result.columns = ['Mother', 'Daughter', 'Total_Network']
        result['movie']=movie
        result['Timepoint'] = p.findall(cell)[-1]
        all_cells=all_cells.append(result)
all_cells['Timepoint'] = all_cells['Timepoint'].astype(int)
all_cells = all_cells.set_index([all_cells['movie'], all_cells['Timepoint'] ])

all_cells=all_cells.sort_index(ascending=True)
all_cells = all_cells.drop("movie", axis="columns")
all_cells.drop('Timepoint', axis='columns')
df = pd.DataFrame(results).T
df['Timepoint']= df['Timepoint'].astype(int)
df = df.set_index([df['movie'], df['Timepoint'] ])
df=df.sort_index(ascending=True)
df=df.drop("Timepoint", axis="columns")
df = df.drop('movie', axis='columns')
df = pd.concat([df,all_cells], axis='columns')




df['Complete_Network'] = (df.Mother + df.Daughter).astype(float)
df.loc[:,'Number_of_Spots'] = df.loc[:,'Number_of_Spots'].astype(int)
df['Segments'] = df['Segments'].astype(int)
df['Point_of_bud_emergence'] = df['Point_of_bud_emergence'].astype(int)
df['volMother'] = df['volMother'].astype(float)
df['volDaughter'] = df['volDaughter'].astype(float)
df['Volume_total'] =df['Volume_total'].astype(np.float32)
df['Timepoint_normalized'] = df['Timepoint_normalized'].astype(int)
df['Nucleoids_in_Mother'] = df['Nucleoids_in_Mother'].astype(int)
df['Nucleoids_in_Daughter'] = df['Nucleoids_in_Daughter'].astype(int)
df['Mean_spot_intensity'] =df['Mean_spot_intensity'].astype(int)
df['Nucleoids_in_both_cells'] =df['Nucleoids_in_both_cells'].astype(int)
df['Spots_per_µm'] =df['Nucleoids_in_both_cells']/df['Complete_Network']
df['Spots_per_µm_Mother']=df['Nucleoids_in_Mother']/df["Mother"]
df['Spots_per_µm_Daughter']=df['Nucleoids_in_Daughter']/df["Daughter"]
df['Network_per_µm^3'] =df['Complete_Network']/df["Volume_total"]
df['Network_per_Volume_Mother']=df['Mother']/df["volMother"]
df['Network_per_Volume_Daughter']= df['Daughter']/df['volDaughter']


df.drop("Timepoint", axis="columns")
#df.drop('movie', axis='columns')
df=df.rename(columns={'Timepoint': 'Timepoints'})

201021_yFT006_series1_000_T0_C=1.tif
201021_yFT006_series1_000_T10_C=1.tif
201021_yFT006_series1_000_T11_C=1.tif
201021_yFT006_series1_000_T12_C=1.tif
201021_yFT006_series1_000_T13_C=1.tif
201021_yFT006_series1_000_T14_C=1.tif
201021_yFT006_series1_000_T15_C=1.tif
201021_yFT006_series1_000_T16_C=1.tif
201021_yFT006_series1_000_T17_C=1.tif
201021_yFT006_series1_000_T1_C=1.tif
201021_yFT006_series1_000_T2_C=1.tif
201021_yFT006_series1_000_T3_C=1.tif
201021_yFT006_series1_000_T4_C=1.tif
201021_yFT006_series1_000_T5_C=1.tif
201021_yFT006_series1_000_T6_C=1.tif
201021_yFT006_series1_000_T7_C=1.tif
201021_yFT006_series1_000_T8_C=1.tif
201021_yFT006_series1_000_T9_C=1.tif
201021_yFT006_series1_000_T0_C=
201021_yFT006_series1_000_T10_C=
201021_yFT006_series1_000_T11_C=
201021_yFT006_series1_000_T12_C=
201021_yFT006_series1_000_T13_C=
201021_yFT006_series1_000_T14_C=
201021_yFT006_series1_000_T15_C=
201021_yFT006_series1_000_T16_C=
201021_yFT006_series1_000_T17_C=
201021_yFT006_series1_000_T1_C

201021_yFT006_series3_003_T13_C=
201021_yFT006_series3_003_T14_C=
201021_yFT006_series3_003_T15_C=
201021_yFT006_series3_003_T16_C=
201021_yFT006_series3_003_T17_C=
201021_yFT006_series3_003_T1_C=
201021_yFT006_series3_003_T2_C=
201021_yFT006_series3_003_T3_C=
201021_yFT006_series3_003_T4_C=
201021_yFT006_series3_003_T5_C=
201021_yFT006_series3_003_T6_C=
201021_yFT006_series3_003_T7_C=
201021_yFT006_series3_003_T8_C=
201021_yFT006_series3_003_T9_C=
201021_yFT006_series3_004_T0_C=1.tif
201021_yFT006_series3_004_T10_C=1.tif
201021_yFT006_series3_004_T11_C=1.tif
201021_yFT006_series3_004_T12_C=1.tif
201021_yFT006_series3_004_T13_C=1.tif
201021_yFT006_series3_004_T14_C=1.tif
201021_yFT006_series3_004_T15_C=1.tif
201021_yFT006_series3_004_T16_C=1.tif
201021_yFT006_series3_004_T17_C=1.tif
201021_yFT006_series3_004_T1_C=1.tif
201021_yFT006_series3_004_T2_C=1.tif
201021_yFT006_series3_004_T3_C=1.tif
201021_yFT006_series3_004_T4_C=1.tif
201021_yFT006_series3_004_T5_C=1.tif
201021_yFT006_series3

C:\Develop\Anaconda3\lib\site-packages\ipykernel_launcher.py:426: RuntimeWarning: divide by zero encountered in double_scalars
C:\Develop\Anaconda3\lib\site-packages\ipykernel_launcher.py:429: RuntimeWarning: invalid value encountered in double_scalars


201021_yFT006_series4_002_T9_C=
201021_yFT006_series5_000_T0_C=1.tif
201021_yFT006_series5_000_T10_C=1.tif
201021_yFT006_series5_000_T11_C=1.tif
201021_yFT006_series5_000_T12_C=1.tif
201021_yFT006_series5_000_T13_C=1.tif
201021_yFT006_series5_000_T14_C=1.tif
201021_yFT006_series5_000_T15_C=1.tif
201021_yFT006_series5_000_T16_C=1.tif
201021_yFT006_series5_000_T17_C=1.tif
201021_yFT006_series5_000_T1_C=1.tif
201021_yFT006_series5_000_T2_C=1.tif
201021_yFT006_series5_000_T3_C=1.tif
201021_yFT006_series5_000_T4_C=1.tif
201021_yFT006_series5_000_T5_C=1.tif
201021_yFT006_series5_000_T6_C=1.tif
201021_yFT006_series5_000_T7_C=1.tif
201021_yFT006_series5_000_T8_C=1.tif
201021_yFT006_series5_000_T9_C=1.tif
201021_yFT006_series5_000_T0_C=
201021_yFT006_series5_000_T10_C=
201021_yFT006_series5_000_T11_C=
201021_yFT006_series5_000_T12_C=
201021_yFT006_series5_000_T13_C=
201021_yFT006_series5_000_T14_C=
201021_yFT006_series5_000_T15_C=
201021_yFT006_series5_000_T16_C=
201021_yFT006_series5_000_T17_C

201021_yFT006_series6_001_T4_C=
201021_yFT006_series6_001_T5_C=
201021_yFT006_series6_001_T6_C=
201021_yFT006_series6_001_T7_C=
201021_yFT006_series6_001_T8_C=
201021_yFT006_series6_001_T9_C=
201023_yFT006_series1_000_T0_C=1.tif
201023_yFT006_series1_000_T10_C=1.tif
201023_yFT006_series1_000_T11_C=1.tif
201023_yFT006_series1_000_T12_C=1.tif
201023_yFT006_series1_000_T13_C=1.tif
201023_yFT006_series1_000_T14_C=1.tif
201023_yFT006_series1_000_T15_C=1.tif
201023_yFT006_series1_000_T16_C=1.tif
201023_yFT006_series1_000_T17_C=1.tif
201023_yFT006_series1_000_T1_C=1.tif
201023_yFT006_series1_000_T2_C=1.tif
201023_yFT006_series1_000_T3_C=1.tif
201023_yFT006_series1_000_T4_C=1.tif
201023_yFT006_series1_000_T5_C=1.tif
201023_yFT006_series1_000_T6_C=1.tif
201023_yFT006_series1_000_T7_C=1.tif
201023_yFT006_series1_000_T8_C=1.tif
201023_yFT006_series1_000_T9_C=1.tif
201023_yFT006_series1_000_T0_C=
201023_yFT006_series1_000_T10_C=
201023_yFT006_series1_000_T11_C=
201023_yFT006_series1_000_T12_C=
201

201023_yFT006_series2_004_T2_C=
201023_yFT006_series2_004_T3_C=
201023_yFT006_series2_004_T4_C=
201023_yFT006_series2_004_T5_C=
201023_yFT006_series2_004_T6_C=
201023_yFT006_series2_004_T7_C=
201023_yFT006_series2_004_T8_C=
201023_yFT006_series2_004_T9_C=
201023_yFT006_series2_005_T0_C=1.tif
201023_yFT006_series2_005_T10_C=1.tif
201023_yFT006_series2_005_T11_C=1.tif
201023_yFT006_series2_005_T12_C=1.tif
201023_yFT006_series2_005_T13_C=1.tif
201023_yFT006_series2_005_T14_C=1.tif
201023_yFT006_series2_005_T15_C=1.tif
201023_yFT006_series2_005_T16_C=1.tif
201023_yFT006_series2_005_T17_C=1.tif
201023_yFT006_series2_005_T1_C=1.tif
201023_yFT006_series2_005_T2_C=1.tif
201023_yFT006_series2_005_T3_C=1.tif
201023_yFT006_series2_005_T4_C=1.tif
201023_yFT006_series2_005_T5_C=1.tif
201023_yFT006_series2_005_T6_C=1.tif
201023_yFT006_series2_005_T7_C=1.tif
201023_yFT006_series2_005_T8_C=1.tif
201023_yFT006_series2_005_T9_C=1.tif
201023_yFT006_series2_005_T0_C=
201023_yFT006_series2_005_T10_C=
20102

201026_yFT006_series1_000_T16_C=
201026_yFT006_series1_000_T17_C=
201026_yFT006_series1_000_T1_C=
201026_yFT006_series1_000_T2_C=
201026_yFT006_series1_000_T3_C=
201026_yFT006_series1_000_T4_C=
201026_yFT006_series1_000_T5_C=
201026_yFT006_series1_000_T6_C=
201026_yFT006_series1_000_T7_C=
201026_yFT006_series1_000_T8_C=
201026_yFT006_series1_000_T9_C=
201026_yFT006_series3_000_T0_C=1.tif
201026_yFT006_series3_000_T10_C=1.tif
201026_yFT006_series3_000_T11_C=1.tif
201026_yFT006_series3_000_T12_C=1.tif
201026_yFT006_series3_000_T13_C=1.tif
201026_yFT006_series3_000_T14_C=1.tif
201026_yFT006_series3_000_T15_C=1.tif
201026_yFT006_series3_000_T16_C=1.tif
201026_yFT006_series3_000_T17_C=1.tif
201026_yFT006_series3_000_T1_C=1.tif
201026_yFT006_series3_000_T2_C=1.tif
201026_yFT006_series3_000_T3_C=1.tif
201026_yFT006_series3_000_T4_C=1.tif
201026_yFT006_series3_000_T5_C=1.tif
201026_yFT006_series3_000_T6_C=1.tif
201026_yFT006_series3_000_T7_C=1.tif
201026_yFT006_series3_000_T8_C=1.tif
201026_yF

201026_yFT006_series4_002_T1_C=
201026_yFT006_series4_002_T2_C=
201026_yFT006_series4_002_T3_C=
201026_yFT006_series4_002_T4_C=
201026_yFT006_series4_002_T5_C=
201026_yFT006_series4_002_T6_C=
201026_yFT006_series4_002_T7_C=
201026_yFT006_series4_002_T8_C=
201026_yFT006_series4_002_T9_C=
201026_yFT006_series6_000_T0_C=1.tif
201026_yFT006_series6_000_T10_C=1.tif
201026_yFT006_series6_000_T11_C=1.tif
201026_yFT006_series6_000_T12_C=1.tif
201026_yFT006_series6_000_T13_C=1.tif
201026_yFT006_series6_000_T14_C=1.tif
201026_yFT006_series6_000_T15_C=1.tif
201026_yFT006_series6_000_T16_C=1.tif
201026_yFT006_series6_000_T17_C=1.tif
201026_yFT006_series6_000_T1_C=1.tif
201026_yFT006_series6_000_T2_C=1.tif
201026_yFT006_series6_000_T3_C=1.tif
201026_yFT006_series6_000_T4_C=1.tif
201026_yFT006_series6_000_T5_C=1.tif
201026_yFT006_series6_000_T6_C=1.tif
201026_yFT006_series6_000_T7_C=1.tif
201026_yFT006_series6_000_T8_C=1.tif
201026_yFT006_series6_000_T9_C=1.tif
201026_yFT006_series6_000_T0_C=
201026

StatisticsError: mean requires at least one data point

In [ ]:
def detect_name(x):
    if "201021" in x:
        return "1st"
    elif "201023" in x:
        return "2nd"
    elif "201026_2" in x:
        return "3rd"
    elif "201026_y" in x:
        return "4th"
    else:
        return "no"

df["replicate"] = df.index.get_level_values(0)
df["replicate"] = df["replicate"].apply(detect_name)

In [2]:

def runMitoGraph(folder):
    '''takes a folder name as input and runs the MitoGraph software on a folder 
    in the subfolder "Channel1" '''
    os.chdir('/home/huygens/Documents/Microscopy')    
    command = 'MitoGraph -xy 0.11 -z 0.2 -path ' + folder + '/Channel1/'
    
    output=subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True)
    os.mkdir(folder + '/Channel1/MitoGraphFiles')
    
    for file in os.listdir(folder + '/Channel1/'):
        if not file.endswith('.tif') and file != 'MitoGraphFiles' and file != "SpotCoordinates":
            shutil.move(folder + '/Channel1/' + file, folder + '/Channel1/MitoGraphFiles/')
            
    
    return output 

    

def getSpotCoordinates(file):
    '''reads .csv file (input) and returns numpy array with coordinates of spots'''
    spot_c = pd.read_csv(file)
    spot_c = np.array(spot_c[['X','Y', 'Z']])
    scale = np.array([0.110, 0.110, 0.200]) # needs to be changed if other microscope is used
    spot_c =  spot_c*scale
        
    return np.stack([spot_c[:,0], spot_c[:,1], spot_c[:,2]], axis=1)
    
def getOutline(file):
    a = pd.read_csv(file)
    a = a[['X', 'Y']]
    a['Z']=0
    a = np.array(a)
    if len(a) == 4:
        volumeMother = calculateVolumeOfCell(np.linalg.norm(a[0]-a[1])/2,
                                             np.linalg.norm(a[2]-a[3])/2,
                                             np.linalg.norm(a[2]-a[3])/2)
        volumeDaughter = 0
    elif len(a) ==8:
        volumeMother = calculateVolumeOfCell(np.linalg.norm(a[0]-a[1])/2,
                                             np.linalg.norm(a[2]-a[3])/2,
                                             np.linalg.norm(a[2]-a[3])/2)
        volumeDaughter = calculateVolumeOfCell(np.linalg.norm(a[4]-a[5])/2,
                                             np.linalg.norm(a[6]-a[7])/2,
                                             np.linalg.norm(a[6]-a[7])/2)
       
    else:
        volumeMother = 'Problem'
        volumeDaughter = 'Problem'
        
    return a, volumeMother, volumeDaughter

def calculateVolumeOfCell(a, b, c):
    return ((4/3)*np.pi*a*b*c)/1000

def getEmergencePoint(file):
    point = list(pd.read_csv(file))

    return point[0]

def closestPoint(a,b,p):
    '''takes two coordinates (a, b) of a line and the coordinates of a point as
    input and returns the coordinates of the point projected on the line'''
    ap = p-a
    ab = b-a
    result = a+ np.dot(ap, ab)/np.dot(ab,ab) * ab
    return result
    

def getFilamentFromVtk(fileName):
    '''takes a filename of a vtk-file (output from MitoGraph) as input and
    returns the filament points and edges and a list containing lists of points
    belonging to separate segments of the mitochondrial network'''

    
    Reader = vtk.vtkPolyDataReader()
    Reader.SetFileName(fileName)
    Reader.Update()
    
    PolyData = Reader.GetOutput()
    
    '''get points of filament'''
    Points = PolyData.GetPoints()
    fil_c = vtk_to_numpy(Points.GetData())
    
    '''Get edges of filament'''
    segments = PolyData.GetNumberOfCells()
    fil_e = np.empty((0,2))
    for i in range(segments):
        for j in range(PolyData.GetCell(i).GetNumberOfPoints()-1):
            fil_e = np.append(fil_e, np.array([[PolyData.GetCell(i).GetPointId(j),PolyData.GetCell(i).GetPointId(j+1)]]),axis=0)
           
    return fil_c, fil_e.astype(int), getSeparateSegments(PolyData)
    
def getSeparateSegments(PolyData):
    '''This function returns a list of lists that consist of points belonging to 
    separate segments of the mitochondrial network'''
    

    
    
    segments = PolyData.GetNumberOfCells()
    
    segmentlist = []    
    
    for i in range(segments):
        segmentlist.append([])
        for j in range(PolyData.GetCell(i).GetNumberOfPoints()):
            segmentlist[i].append(PolyData.GetCell(i).GetPointId(j))

    def to_graph(l):
        G = networkx.Graph()
        for part in l:
            # each sublist is a bunch of nodes
            G.add_nodes_from(part)
            # it also imlies a number of edges:
            G.add_edges_from(to_edges(part))
        return G
    
    def to_edges(l):
        """ 
            treat `l` as a Graph and returns it's edges 
            to_edges(['a','b','c','d']) -> [(a,b), (b,c),(c,d)]
        """
        it = iter(l)
        last = next(it)
    
        for current in it:
            yield last, current
            last = current    
    
    G = to_graph(segmentlist)
    return [list(i) for i in list(connected_components(G))]
    
def getSpotDistributionOnAxis(axisOrigin, axisEnd, spotCoordinates):
    distances = np.array([np.linalg.norm(axisOrigin-closestPoint(axisOrigin, axisEnd, i)) for i in spotCoordinates])
    motherCellDistribution = distances[distances<np.linalg.norm(axisOrigin-axisEnd)]
    spotsInMother = len(motherCellDistribution)
    spotsInDaughter = len(distances) -  spotsInMother
    relativePositionOnAxis = motherCellDistribution/np.linalg.norm(axisOrigin-axisEnd)
    return np.sort(relativePositionOnAxis), spotsInMother, spotsInDaughter
    

def main(rootFolder):
    cellDict ={}
    mitoGraphOutput = runMitoGraph(rootFolder + '/Channel_1/')
    
    for file in os.listdir(rootFolder + '/Cell_outline'):
        channels = ['2', '3']       
        cellID = file[:-6] 
        cellDict[cellID] = {}
        (cellDict[cellID]['Outline'],
         cellDict[cellID]['Volume Mother Cell'],
         cellDict[cellID]['Volume Daughter Cell']) = getOutline(rootFolder + '/Cell_outline/' + file)
        
        (cellDict[cellID]['Filament coordinates'],
         cellDict[cellID]['Filament edges'],
         cellDict[cellID]['Separate segments']) = getFilamentFromVtk(rootFolder + '/Channel_1/' + cellID + '_1_skeleton.vtk')
        '''If the outline.csv filr for a cell contains 4 rows, it is only one cell 
        without a daughter, if it contains 8 rows, it has a daughter'''
        
        if len(cellDict[cellID]['Outline']) == 8:
            cellDict[cellID]['Daughter Cell'] = True
        else:
            cellDict[cellID]['Daughter Cell'] = False
        
        for i in channels:
            
            channelVar = 'Channel_' + i + ' '
            cellDict[cellID][channelVar + 'Spots'] = getSpotCoordinates(rootFolder + '/Channel_' +
                                                                        i + '/SpotCoordinates/'
                                                                        + cellID + '_' + i + '.csv')
            
            (cellDict[cellID][channelVar + 'Spot distribution'],
            cellDict[cellID][channelVar + 'Spots in Mother Cell'],
            cellDict[cellID][channelVar + 'Spots in Daughter Cell']) = getSpotDistributionOnAxis(cellDict[cellID]['Outline'][0],
                                                                     cellDict[cellID]['Outline'][1],
                                                                     cellDict[cellID][channelVar + 'Spots'])
             
       
    
    return cellDict, mitoGraphOutput
    
        

def closestDistance(fil_c, fil_e, colocSpot_1, colocSpot_2):
    
    spotSummary = {}
    spotSummary['shortestDistances'] = []
    for i in range(len(colocSpot_1)):
        
        branches = []
        deadEnds = []
        
        startEndDistance = []
        
        #the following if statement determines whether spots in the microscopy images colocalize
        if colocSpot_1[i] in colocSpot_2:
            startEndDistance = [[colocSpot_1[i], colocSpot_1[i], 0]]
        else:
            for element in fil_e[(fil_e==colocSpot_1[i]).sum(1).astype(bool)]: #finds the rows in the edges array that contain the spot
                                                                             #under consideration
                
                #the next line puts the spot under consideration and the next spot 
                #it connects to in every direction into the branches list            
                branches.append([[colocSpot_1[i], element[np.not_equal(element, colocSpot_1[i])][0]]])
          
            for branchNumber in range(len(branches)): #loops through branches 
                
                while any(q[-1] not in colocSpot_2 for q in branches[branchNumber]):
                    
                    indeces=[]
                    for subBranchNumber, subBranch in enumerate(branches[branchNumber]): #loops through possible subbranches of branch.
                                                                        #in the first loop only one subbranch is present
                        if subBranch[-1] not in colocSpot_2:
                                                         
                            #next line returns the edges that to the last point in the subbranch
                            #it however excludes edges to points already in the subBranch this
                            #prevents loops
                            nextEdge = fil_e[(fil_e==subBranch[-1]).sum(1).astype(bool)
                            & ~(sum([fil_e==q for q in subBranch[:-1]])).sum(1).astype(bool)]
                            
                            #if there is only one new point it connects to 
                                                    
                            if nextEdge.shape[0]==1:
                                   
                                #next line appends next point to subbranch
                                branches[branchNumber][subBranchNumber].append(
                                nextEdge[np.not_equal(nextEdge, subBranch[-1])][0])
                            
                            #if the spot under consideration branches
                            elif nextEdge.shape[0]>=1:
                                                            
                                for j in range(nextEdge.shape[0]-1):
                                    
                                    branches[branchNumber].insert(subBranchNumber,subBranch.copy())
                                    lastPoint = subBranch[-1]
        
                                #the following loop appends points to subbranches
                                for j in range(nextEdge.shape[0]):
                                    
                                    branches[branchNumber][subBranchNumber+j].append(
                                    nextEdge[j][np.not_equal(nextEdge[j], lastPoint)][0])
                                break
                            else: #no point connects to last point, delete subbranch and put
                                  #into shortBranches list
                                
                                indeces.append(subBranchNumber)
                    indeces.reverse()
                    for k in range(len(indeces)):
                        deadEnds.append(branches[branchNumber].pop(indeces[k]))
                    
                if len(branches[branchNumber]) > 0: 
                                       
                    for branch2 in branches[branchNumber]:
                        if calculateDistance(branch2, fil_c) == min([calculateDistance(q, fil_c) for q in branches[branchNumber]]):
                            startEndDistance.extend([sorted([branch2[0], branch2[-1]])+[calculateDistance(branch2, fil_c)]])                        
                
        startEndDistance=np.asarray(startEndDistance)
        if startEndDistance.size == 3:
            try:
                spotSummary['shortestDistances'].append(startEndDistance[0,2])
            except:
                print(str(i))
        elif startEndDistance.size >= 3:
            spotSummary['shortestDistances'].append(startEndDistance[:,2].min())
            
        spotSummary['Spot#' + str(i)] = {'branches':branches, 'deadEnds':deadEnds, 'Distances':startEndDistance}
        
    return spotSummary
    
def calculateDistance(segment, fil_c):
    
    tempEdges = np.array([segment[:-1], segment[1:]])
    return np.sqrt(((fil_c[tempEdges[0, :]] - fil_c[tempEdges[1, :]])**2).sum(1)).sum()
    
def colocalizer(spots, fil_c):
    '''Takes the coordinates of Spots and the coordinates of the filaments as input
    and calculates the distance from the each spot to every point in the filament
    and returns an array that contains the indices of the points on the filament
    that are closest to the spots'''
       
    colocSpot = []
     
    # loops through all spots
    for j in range(spots.shape[0]):
    
        # generates array that holds the distance of all points on the filament
        # to the point under consideration    
        
        distances = np.sqrt(((fil_c - spots[j])**2).sum(1))
        
        # adds the index of the spot on the filament that is closest to the 
        # spot to the colocSpot list that is returned by the function
        colocSpot.append(int(np.where(distances == min(distances))[0][0]))    
     
        # the following line gets the coordinates of the spots on the filament that are closest
        # to the spots. The function also returns this array as the second value'''      
        
        colocSpotCoord = fil_c[colocSpot]        
        
    return colocSpot, colocSpotCoord

def colocplotter(filCoordinates, redSpots, greenSpots):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
      
    #specialSpot = int(input('Please enter filament spot, which should be hoghlighted:'))

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    Axes3D.scatter(ax, redSpots[:,0], redSpots[:,1], zs=redSpots[:,2], c='r', s=30)
    Axes3D.scatter(ax, greenSpots[:,0], greenSpots[:,1], zs=greenSpots[:,2], c='g', s=30)   
    #Axes3D.scatter(ax, colocSpotsCoord[:,0], colocSpotsCoord[:,1], c='g', zs=colocSpotsCoord[:,2], s=40)
    #Axes3D.scatter(ax, filCoordinates[specialSpot,0], filCoordinates[specialSpot,1], c='k', zs=filCoordinates[specialSpot,2], s=100)
    
    #the following line removes the coordinates from the filament, where
    #colocalizing spots and the special spot are
    filCoordinates = np.delete(filCoordinates, np.append(redSpots, greenSpots), 0)     
    
    Axes3D.scatter(ax, filCoordinates[:,0], filCoordinates[:,1], zs=filCoordinates[:,2])

def generateRandomColocalizedSpots(filament, numberOfSpots):
    return random.sample(list(range(filament[0].shape[0])), numberOfSpots)


def main3(folder):
    
    cellDict = {}
    for cell in os.listdir(folder):
        if cell.endswith('_skeleton.vtk'):
            cellDict[cell] ={}
            fil_c, fil_e, segments = getFilamentFromVtk(cell)
            cellDict[cell]['Coordinates'] = fil_c 
            cellDict[cell]['Edges'] = fil_e
            cellDict[cell]['Segments'] = segments 
            cellDict[cell]['Network Lenght'] = np.array([calculateDistance(x, fil_c) for x in segments]).sum()





def main_(folder):
    cellSummary = {} 
    cellSummary['AllDistancesGreenToRed']=[]
    cellSummary['AllDistancesRedToGreen']=[]
    cellSummary['AllDistancesRandomGreenToRed'] = []
    cellSummary['AllDistancesRandomRedToGreen'] = []
    runMitoGraph(folder)
        
    fileList = [file for file in os.listdir(folder + 'Channel_3') if file.endswith('.tif')]
    
    for file in fileList:
        cellSummary[file] = {}
        vtkFile = folder + 'Channel_3/MitoGraphFiles/' + file.replace('.tif', '_skeleton.vtk')
       
        redSpotFile = folder + 'Channel_1/SpotCoordinates/' + file.replace('tif', 'csv')
        redSpotFile = redSpotFile.replace('_3_', '_1_')
        greenSpotFile = folder + 'Channel_2/SpotCoordinates/' + file.replace('tif', 'csv') 
        greenSpotFile = greenSpotFile.replace('_3_', '_2_')
        cellSummary[file]['Filament'] = getFilamentFromVtk(vtkFile)
        cellSummary[file]['RedSpots'] = getSpotCoordinates(redSpotFile)
        (cellSummary[file]['RedSpotsColocSpot'],
        cellSummary[file]['RedSpotsColocCoord'])= colocalizer(
                                                 cellSummary[file]['RedSpots'],
                                                 cellSummary[file]['Filament'][0])
        
        cellSummary[file]['GreenSpots'] = getSpotCoordinates(greenSpotFile)                                         
        (cellSummary[file]['GreenSpotsColocSpot'],
        cellSummary[file]['GreenSpotsColocCoord'])= colocalizer(
                                                 cellSummary[file]['GreenSpots'],
                                                 cellSummary[file]['Filament'][0])
                                                 
        cellSummary[file]['randomGreenSpots'] = generateRandomColocalizedSpots(
                                                cellSummary[file]['Filament'],
                                                cellSummary[file]['GreenSpots'].shape[0])
        cellSummary[file]['randomRedSpots'] = generateRandomColocalizedSpots(
                                              cellSummary[file]['Filament'],
                                              cellSummary[file]['RedSpots'].shape[0])        
        
        cellSummary[file]['DistanceOfGreenToRed'] = closestDistance(
                                                    cellSummary[file]['Filament'][0],
                                                    cellSummary[file]['Filament'][1],
                                                    cellSummary[file]['GreenSpotsColocSpot'],
                                                    cellSummary[file]['RedSpotsColocSpot'])
        
        cellSummary[file]['DistanceOfRedToGreen'] = closestDistance(
                                                    cellSummary[file]['Filament'][0],
                                                    cellSummary[file]['Filament'][1],
                                                    cellSummary[file]['RedSpotsColocSpot'],
                                                    cellSummary[file]['GreenSpotsColocSpot'])
                                                    
        cellSummary['AllDistancesGreenToRed'] += cellSummary[file]['DistanceOfGreenToRed']['shortestDistances']    
        cellSummary['AllDistancesRedToGreen'] += cellSummary[file]['DistanceOfRedToGreen']['shortestDistances'] 
        
        cellSummary[file]['DistanceOfRandomGreenToRed'] = closestDistance(
                                                    cellSummary[file]['Filament'][0],
                                                    cellSummary[file]['Filament'][1],
                                                    cellSummary[file]['randomGreenSpots'],
                                                    cellSummary[file]['RedSpotsColocSpot'])
        
        cellSummary[file]['DistanceOfRandomRedToGreen'] = closestDistance(
                                                    cellSummary[file]['Filament'][0],
                                                    cellSummary[file]['Filament'][1],
                                                    cellSummary[file]['randomRedSpots'],
                                                    cellSummary[file]['GreenSpotsColocSpot'])
                                                    
        cellSummary['AllDistancesGreenToRed'] += cellSummary[file]['DistanceOfGreenToRed']['shortestDistances']    
        cellSummary['AllDistancesRedToGreen'] += cellSummary[file]['DistanceOfRedToGreen']['shortestDistances']
        cellSummary['AllDistancesRandomGreenToRed'] += cellSummary[file]['DistanceOfRandomGreenToRed']['shortestDistances']    
        cellSummary['AllDistancesRandomRedToGreen'] += cellSummary[file]['DistanceOfRandomRedToGreen']['shortestDistances']                                 
    
    labels = ['AllDistancesGreenToRed', 'AllDistancesRedToGreen', 'AllDistancesRandomGreenToRed', 'AllDistancesRandomRedToGreen']                                     
    data = [cellSummary[sample] for sample in labels]    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    bins = np.linspace(0,3,10)
    
    ax.hist(data, bins=bins, normed=True, label = labels)
    ax.legend()
    
    
    return cellSummary

def convertIntoPixelUnit(file, xydim=0.11, zdim=0.2):
    df = pd.read_table( file +  '1.txt')
    # the following converts the µm unit into pixel unit
    df['alt_x'] = round(df['x']/xydim).astype('int32')
    df['alt_y'] =200- round(df['y']/xydim).astype('int32')
    df['alt_z'] = round(df['z']/zdim).astype('int32')
    return df

def defineCell(df, mask, mask_d):
    a = []
    for i in range(df.shape[0]):
        if type(mask_d)== int:
            if mask[df.loc[i, 'alt_y'], df.loc[i, 'alt_x']]:
                a.append('Mother')
            else:
                a.append('None')
        else:
            if mask[df.loc[i, 'alt_y'], df.loc[i, 'alt_x']]:
                a.append('Mother')
            elif mask_d[df.loc[i, 'alt_y'], df.loc[i, 'alt_x']]:
                a.append('Daughter')
            else:
                a.append('None')
    return a

def calculateNetworkLength(df):
    wholeNetwork = []
    uniqueLines = set(df['line_id'])
    for uniqueLine in uniqueLines:
        df2 = df[df['line_id']==uniqueLine] # iterate through lines
        summary = []
        for i in range(df2.shape[0]-1):
            a = np.array(df2.iloc[i,2:5])
            b = np.array(df2.iloc[i+1,2:5])
            summary.append(np.linalg.norm(a-b))
        wholeNetwork.append(sum(summary))
    return sum(wholeNetwork)
   
def slope(x1, y1, x2, y2): # Line slope given two points:
    if x1==x2:
        return (y2-y1)/((x2+0.0001)-x1)
    else:
        return (y2-y1)/(x2-x1)

def angle(s1, s2): #calculate the angle of the slope function output
    angle_=math.degrees(math.atan((s2-s1)/(1+(s2*s1))))

    return angle_


def line(p1, p2): #draws a line between two coordinates. lines serve as input for intersection function
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2): #finds the intersection between two lines
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False

def calculateTheDistance(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist  

def getSpotCoordinates_mask(file): #returns x,y spotcoordinates without scaling them to microscope settings
    '''reads .csv file (input) and returns numpy array with coordinates of spots'''
    spot_c = pd.read_csv(file)
    spot_c = np.array(spot_c[['X','Y','V']]) 
    return np.stack([spot_c[:,0], spot_c[:,1],spot_c[:,2]], axis=1)

def assignNucleoids(file, mask, mask_d): #assigns the coordinates of getSpotCoordinates_mask function to either 
                           #mother mask or daughter mask
    nucleoids_mother=[]
    nucleoids_daughter=[]
    OutsideOfMask=[]
    Intensities=[]
    for spot in file:
        if type(mask_d)== int:
            if mask[spot[1],spot[0]]==1:
                nucleoids_mother.append(spot)
                Intensities.append(spot[2])
            else:
                OutsideOfMask.append(spot)
        else:
            if mask[spot[1],spot[0]]==1:
                nucleoids_mother.append(spot)
                Intensities.append(spot[2])
            elif mask_d[spot[1],spot[0]]==1:
                nucleoids_daughter.append(spot)
                Intensities.append(spot[2])
            else:
                OutsideOfMask.append(spot)
    return np.array(nucleoids_mother), np.array(nucleoids_daughter), np.array(OutsideOfMask), np.array(Intensities)

def assignNetwork(file, mask, mask_d): #assigns the coordinates of the Network (fil_c) function to either 
                                       #mother mask or daughter mask
    scale = np.array([0.110, 0.110, 0.200])
    file = np.around(file/scale).astype(int)
    file[:,1] = 200-file[:,1]
    Network_mother=[]
    Network_daughter=[]
    OutsideOfMask=[]
    for Coordinate in file:
        if type(mask_d)== int:
            if mask[Coordinate[1],Coordinate[0]]==1:
                Network_mother.append(Coordinate)
            else:
                OutsideOfMask.append(Coordinate)
        else:
            if mask[Coordinate[1],Coordinate[0]]==1:
                Network_mother.append(Coordinate)
              
            elif mask_d[Coordinate[1],Coordinate[0]]==1:
                Network_daughter.append(Coordinate)
            else:
                OutsideOfMask.append(Coordinate)
    return np.array(Network_mother), np.array(Network_daughter), np.array(OutsideOfMask)

def percentageNetwork(Network_mother, Network_daughter, OutsideOfMask):
    
    relNetworkMother = len(Network_mother)/(len(Network_mother)+len(Network_daughter)+len(OutsideOfMask))
    relNetworkDaughter = len(Network_daughter)/(len(Network_mother)+len(Network_daughter)+len(OutsideOfMask))

    return relNetworkMother, relNetworkDaughter


def getEllipses(file): #creates masks of BF cells
    a = pd.read_csv(file)
    a = a[['X', 'Y']]
    a= a.astype(int)
    '''
    If the outline.csv filr for a cell contains 4 rows, it is only one cell 
    without a daughter, if it contains 8 rows, it has a daughter
    '''
    if len(a) == 4:
        #define all parameters needed for draw.ellipse command:
        height = calculateTheDistance(a["X"][0],a["Y"][0],a["X"][1],a["Y"][1])
        width = calculateTheDistance(a["X"][2],a["Y"][2],a["X"][3],a["Y"][3])
        L1 = line(a.iloc[0], a.iloc[1])
        L2 = line(a.iloc[2],a.iloc[3])
        middle=intersection(L1,L2)
        #get ellipse
        rr,cc =  draw.ellipse(middle[0],middle[1], (height/2), (width/2) )
        
        #create a mask by defining all points of the ellipse in an empty 
        #image with the parameters of the BF picture as 1. coordinates outside the ellipse = 0
        mask= np.zeros((200,200))
        mask[cc,rr] = True
        mask = skimage.morphology.dilation(mask, selem=None, out=None, shift_x=False, shift_y=False)
        #rotate the mask. needed parameters: angle from slope and angle functions. 
        #center= middle from line and intersection function 
        
        slope1=slope(a['X'][0],a['Y'][0],a['X'][1],a['Y'][1])
        slope2=slope(0,int(middle[1]),199,int(middle[1])) #WARNING: slope2 is defined as the x axis going from coordinate (0,0), 
                                #to (199,0). If you use pictures with another size than 200x200, change the values
        _angle = angle(slope1,slope2)
        mask=rotate(mask,angle=_angle, center = middle)
        
        mask_d=0
        height_d=0
        width_d=0
        middle_d=0
        slope3=0
        slope4=0
        
    elif len(a) == 8: 
        height = calculateTheDistance(a["X"][0],a["Y"][0],a["X"][1],a["Y"][1])
        width = calculateTheDistance(a["X"][2],a["Y"][2],a["X"][3],a["Y"][3])
        L1 = line(a.iloc[0], a.iloc[1])
        L2 = line(a.iloc[2],a.iloc[3])
        middle=intersection(L1,L2)
        rr,cc =  draw.ellipse(middle[0],middle[1], (height/2), (width/2) )
        
        
        
        
        height_d=calculateTheDistance(a["X"][4],a["Y"][4],a["X"][5],a["Y"][5])
        width_d=calculateTheDistance(a["X"][6],a["Y"][6],a["X"][7],a["Y"][7])
        L1_d = line(a.iloc[4], a.iloc[5])
        L2_d = line(a.iloc[6],a.iloc[7])
        middle_d = intersection(L1_d,L2_d)
        dd,ff = draw.ellipse(middle_d[0],middle_d[1], (height_d/2), (width_d/2) )
        
        
        mask= np.zeros((200,200))
        mask[ cc,rr] = True
        mask = skimage.morphology.dilation(mask, selem=None, out=None, shift_x=False, shift_y=False)
                
        slope1=slope(a['X'][0],a['Y'][0],a['X'][1],a['Y'][1])
        slope2=slope(0,int(middle[1]),199,int(middle[1]))
        _angle = angle(slope1,slope2)
        
        
        mask=rotate(mask,angle=_angle, center = middle)
        
        
        mask_d= np.zeros((200,200))
        mask_d[ ff,dd] = True
        mask_d = skimage.morphology.dilation(mask_d, selem=None, out=None, shift_x=False, shift_y=False)
        
                
        slope3=slope(a['X'][4],a['Y'][4],a['X'][5],a['Y'][5]) 
        slope4=slope(0,int(middle_d[1]),199,int(middle_d[1]))
        _angle_d = angle(slope3,slope4)
        mask_d=rotate(mask_d,angle=_angle_d, center = middle_d)
        
    else:
        mask ="Problem"
        mask_d = "Problem"
        
        
    return mask, mask_d